In [1]:
import selenium
from selenium import webdriver
import pandas as pd 
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
import time 
import re

In [2]:
# Create driver
driver = webdriver.Chrome(ChromeDriverManager().install())

# Get the page url 
page_url = "https://transcripts.foreverdreaming.org/viewtopic.php?f=22&t=4411"
driver.get(page_url) # Open the page 
wait = WebDriverWait(driver, 10) # Wait until the page renders

/var/folders/_b/l_819g5924d_y00npn9gt60h0000gn/T/ipykernel_29310/2381540048.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [3]:
item_links = [item.get_attribute("href") for item in driver.find_elements(By.CLASS_NAME, 'postlink')] # Get links to the scripts 

In [11]:
for item_link in item_links[132:154]:
    driver.get(item_link) # Click the link  
    time.sleep(3) 
    l = driver.find_elements(By.TAG_NAME, 'p')
    for i in l: 
        print (i.text) 


F.D. » Transcripts » G » Gilmore Girls
LOGAN'S APARTMENT
[Rory wakes up, walks past some of the mess from the previous nights party and see a gift on the counter. She opens it and is surprised, she pulls or a 2 foot high rocket and stands it on the counter and wonders what it means]
CHRISTOPHER'S APARTMENT – BEDROOM
[Lorelai and Chris are in bed with Chris' arm resting over Lorelai]
CHRISTOPHER: [Grunts] Hey.
LORELAI: Hey.
CHRISTOPHER: What are you doing?
LORELAI: Um...go back to sleep. [Lorelai get up]
CHRISTOPHER: Come back to bed.
LORELAI: Oh, I can't I got to go.
CHRISTOPHER: Stay.
LORELAI: No um I can't -- Paul Anka.
CHRISTOPHER: No way he's performing this early. Check your tickets.
LORELAI: No, I got to go home and feed him.
CHRISTOPHER: Well, let me make you breakfast first.
LORELAI: No.
CHRISTOPHER: Come on you're no good if you collapse from hunger before you feed him. It's like how, when you're on an airplane, you put on your own oxygen mask before your kids. It seems selfis